## KNN 연습문제

### 인도의 간질환자 예측 모델  
- 사용 데이터 세트 : 인도의 간질환자 데이터 (indian_liver_patient.csv)
- 데이터 세트 분리 : 테스트용(평가용) 데이터 세트 
- 사용하는 ML 알고리즘 : KNN 알고리즘  
- 모델 평가 : 예측 성능 평가   
    - 오차행렬, 정확도, 정밀도, 재현율, F1 스코어, ROC AUC 평가 지표 

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
import pandas as pd
import numpy as np

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# 한글 문제
# matplotlib의 기본 폰트에서 한글이 지원되지 않기 때문에
# matplotlib의 폰트 변경 필요
import platform

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

if platform.system() == 'Darwin':  # 맥OS 
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':  # 윈도우
    path = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system...  sorry~~~')

In [5]:
liver_df = pd.read_csv('../../data/indian_liver_patient.csv')
liver_df.head()

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,Female,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,Male,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,Male,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,Male,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,Male,3.9,2.0,195,27,59,7.3,2.4,0.40,1


In [8]:
# 수행해야 될 작업 알아서 처리

- Age
- Gender
- Total_Bilirubin
- Direct_Bilirubin
- Alkaline_Phosphotase
- Alamine_Aminotransferase
- Aspartate_Aminotransferase
- Total_Protiens
- Albumin
- Albumin_and_Globulin_Ratio
- Dataset

## 1. 데이터 준비

### 결측치 처리

In [10]:
# 결측치 확인
liver_df.info()
# Albumin_and_Globulin_Ratio null 4개

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 583 entries, 0 to 582
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         583 non-null    int64  
 1   Gender                      583 non-null    object 
 2   Total_Bilirubin             583 non-null    float64
 3   Direct_Bilirubin            583 non-null    float64
 4   Alkaline_Phosphotase        583 non-null    int64  
 5   Alamine_Aminotransferase    583 non-null    int64  
 6   Aspartate_Aminotransferase  583 non-null    int64  
 7   Total_Protiens              583 non-null    float64
 8   Albumin                     583 non-null    float64
 9   Albumin_and_Globulin_Ratio  579 non-null    float64
 10  Dataset                     583 non-null    int64  
dtypes: float64(5), int64(5), object(1)
memory usage: 50.2+ KB


In [48]:
# Albumin_and_Globulin_Ratio null 4개 처리
# 평균 값 0.9470639032815197 으로 채워준다.
liver_df['Albumin_and_Globulin_Ratio'].fillna(liver_df['Albumin_and_Globulin_Ratio'].mean(), inplace = True)

liver_df['Albumin_and_Globulin_Ratio'].isnull().sum() # 처리 완료

0

### 0 값 처리

In [49]:
# 추가로 0값이 있는지 확인
# liver_df['Age'].value_counts()
# liver_df['Gender'].value_counts()
# liver_df['Total_Bilirubin'].value_counts()
# liver_df['Direct_Bilirubin'].value_counts()
# liver_df['Alkaline_Phosphotase'].value_counts()
# liver_df['Alamine_Aminotransferase'].value_counts()
# liver_df['Aspartate_Aminotransferase'].value_counts()
# liver_df['Total_Protiens'].value_counts()
# liver_df['Albumin'].value_counts()
# liver_df['Albumin_and_Globulin_Ratio'].value_counts()

# 비이상적인 0값 없음

### 레이블 인코딩

In [50]:
# label encoding 적용할 피쳐
# Gender값 문자열 -> 숫자 변경
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder = encoder.fit(liver_df['Gender'])
liver_df['Gender'] = encoder.transform(liver_df['Gender'])

liver_df.head()
# Gender 값이 0, 1로 변경

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio,Dataset
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90,1
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74,1
2,62,1,7.3,4.1,490,60,68,7.0,3.3,0.89,1
3,58,1,1.0,0.4,182,14,20,6.8,3.4,1.00,1
4,72,1,3.9,2.0,195,27,59,7.3,2.4,0.40,1


### 필요없는 열 제거

In [51]:
# 필요없는 열이 발견되지 않음

### label data, feature data

In [40]:
# label data 확인 : 'Dataset'
# class 결정값 1, 2
liver_df['Dataset'].value_counts()

Dataset
1    416
2    167
Name: count, dtype: int64

In [41]:
# label data
label_data = liver_df['Dataset']

# feature data
feature_data = liver_df.drop('Dataset', axis=1)
feature_data.head(2)

,Age,Gender,Total_Bilirubin,Direct_Bilirubin,Alkaline_Phosphotase,Alamine_Aminotransferase,Aspartate_Aminotransferase,Total_Protiens,Albumin,Albumin_and_Globulin_Ratio
0,65,0,0.7,0.1,187,16,18,6.8,3.3,0.90
1,62,1,10.9,5.5,699,64,100,7.5,3.2,0.74


## 2. 데이터 분리 

In [24]:
from sklearn.model_selection import train_test_split

In [42]:
# stratify= label 데이터 옵션 없는 경우
X_train, X_test, y_train, y_test = train_test_split(feature_data,
                                                    label_data,
                                                    test_size=0.2, 
                                                    random_state=11)


In [43]:
# stratify= label 데이터 옵션 없는 경우
# 분리 후 y 값들
y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)

Dataset
1    0.714592
2    0.285408
Name: proportion, dtype: float64

Dataset
1    0.709402
2    0.290598
Name: proportion, dtype: float64

In [44]:
# stratify= label 데이터 옵션 있는 경우
X_train, X_test, y_train, y_test = train_test_split(feature_data,
                                                    label_data,
                                                    test_size=0.2, 
                                                    random_state=11,
                                                    stratify = label_data)


In [59]:
# stratify= label 데이터 옵션 있는 경우
# 분리 전
label_data.value_counts(normalize=True)

# 분리 후 y 값들
y_train.value_counts(normalize=True)
y_test.value_counts(normalize=True)

Dataset
1    0.713551
2    0.286449
Name: proportion, dtype: float64

Dataset
1    0.714592
2    0.285408
Name: proportion, dtype: float64

Dataset
1    0.709402
2    0.290598
Name: proportion, dtype: float64

In [ ]:
# 결과
# stratify 없으나 있으나 똑같음

## 3. 모델링



In [46]:
from sklearn.metrics import accuracy_score

###  알고리즘
- 로지스틱 회귀
- 랜덤 포레스트
- 앙상블 학습
- K-NN
- 결정트리


In [52]:

# 3개의 알고리즘을 사용해서 학습/예측/평가 수행
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier



# 모델 생성
dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(max_iter=1000)
kn_clf = KNeighborsClassifier() # n_neighbors=5 디폴트

In [54]:
# 모델 학습 및 예측
# (1) decision tree
dt_clf.fit(X_train, y_train)
y_pred_dt = dt_clf.predict(X_test)
print('DecisionTreeClassifier 정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred_dt)))


# (2) ensemble
rf_clf.fit(X_train, y_train)
y_pred_rf = rf_clf.predict(X_test)
print('RandomForestClassifier 정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred_rf)))


# (3) 로지스틱
lr_clf.fit(X_train, y_train)
y_pred_lr = lr_clf.predict(X_test)
print('LogisticRegression 정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred_lr)))

# (4) k-NN
kn_clf.fit(X_train, y_train)
y_pred_kn = kn_clf.predict(X_test)
print('K-NN 정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred_kn)))

DecisionTreeClassifier(random_state=11)

DecisionTreeClassifier 정확도 : 0.6325


RandomForestClassifier(random_state=11)

RandomForestClassifier 정확도 : 0.7009


LogisticRegression(max_iter=1000)

LogisticRegression 정확도 : 0.7521


KNeighborsClassifier()

K-NN 정확도 : 0.6667


In [56]:
# 모델 학습 결과
# RandomForest 의 정확도가 가장 높게 나왔다.

### 예측

In [ ]:
# 임의의 테스트 데이터로 예측
# 훈련된 모델에 테스트 수행
# test_data = []
# kn_clf.predict([test_data])


## 4. 성능 평가 지표 출력